# `LibreASR`

## Imports

In [ ]:
%reload_ext autoreload
%autoreload 3
# %matplotlib inline

In [ ]:
from lib.imports import *

## `Config`

In [ ]:
conf, lang, builder_train, builder_valid, db, m, learn = parse_and_apply_config()
(conf, lang, db, m, learn)

In [ ]:
# builder.plot()

In [ ]:
builder_train.print()
builder_valid.print()

In [ ]:
tpl = db.one_batch()
X, Ym, _, _ = tpl[0]
Y, Y_lens, X_lens = tpl[1]
what(X), what(X_lens), what(Y), what(Y_lens)

In [ ]:
X_lens, Y_lens

### `Statistics`

In [ ]:
# db.statistics(conf["norm_file"], batches=2)

### `Augmentation`

In [ ]:
# db.aug(n=5, only=7)

## `Model`

In [ ]:
# switch to gpu mode
# m.convert_to_gpu()

In [ ]:
m = m.to(conf["cuda"]["device"])

In [ ]:
print("n_params:", n_params(m))

Timing

In [ ]:
%%time
_in = db.one_batch()[0]

In [ ]:
%%time
with torch.no_grad():
    _out = m(cudaize(_in))
    print("X", _in[0].shape, "->", _out.shape)
    print("Y", _in[1].shape)

## `Train`

In [ ]:
ckpt = "model"

In [ ]:
try:
    learn.load(ckpt, with_opt=True, strict=False)
    pass
except Exception as e:
    print("failed loading ckpt", ckpt)
    print(e)

### `fit`

In [ ]:
learn.unfreeze()

In [ ]:
# learn.lr_find()

In [ ]:
# %pdb
# stage-1
eps = 15
lr = 1e-3
wd = 0.1 # 0.01
warmup_epochs = 1.5
div_final = 5.0
learn.fit_one_cycle(eps, lr, wd=wd, cbs=learn.extra_cbs, div_final=div_final, div=100., pct_start=(1./eps)*warmup_epochs)

In [ ]:
# stage-2 (continue)
# freeze encoder?
# learn.freeze_to(1)
eps = 50
lr = 3e-4
wd = 0.01
learn.fit_one_cycle(eps, lr, wd=wd, cbs=learn.extra_cbs, div_final=1000., div=100., pct_start=0.0001)

In [ ]:
# finetuning
eps = 30
lr = 4e-4
learn.fit_one_cycle(eps, lr, cbs=learn.extra_cbs, div_final=500., div=100., pct_start=0.9/eps)

In [ ]:
# testing
eps = 20
lr = 4e-4
learn.fit(eps, lr, cbs=learn.extra_cbs)

### `checks`

In [ ]:
learn.recorder.values

In [ ]:
# save model for later
# learn.save(checkpoints[-1], with_opt=True)
learn.save(ckpt, with_opt=False)

## `Decode`

In [ ]:
# final
_ = list(learn.test(min_samples=256*4, train=False))

In [ ]:
# model
_ = list(learn.test(min_samples=256*4, train=False))

In [ ]:
m.joint.joint[2].bias

In [ ]:
m.joint.joint[2].bias

## Archive

In [ ]:
from lib.model_utils import *
from pathlib import Path

In [ ]:
save_asr_model()